In [1]:
import os
print os.getcwd()

/abaghel/projects/rnn-patient-sentiment-paper


## Prototype the Training 

In [2]:
import cPickle
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Merge
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta
from keras.constraints import unitnorm, maxnorm
from keras.regularizers import l2
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import sequence

from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU

from sklearn.metrics import roc_auc_score

# This import to resolve some errors with tf version on office server
import tensorflow as tf
tf.python.control_flow_ops = tf

Using TensorFlow backend.


### Define data preparation functions


In [3]:
def get_idx_from_sent(sent, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    x = []
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    while len(x) < max_l:
        x.append(0)
    return x

def make_idx_data(revs, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    train, val, test = [], [], []
    for rev in revs:
        sent = get_idx_from_sent(rev['text'], word_idx_map, max_l, kernel_size)
        sent.append(rev['y'])
        if rev['split'] == 1:
            train.append(sent)
        elif rev['split'] == 0:
            val.append(sent)
        else:
            test.append(sent)
    train = np.array(train, dtype=np.int)
    val = np.array(val, dtype=np.int)
    test = np.array(test, dtype=np.int)
    return [train, val, test]

### Load data 

In [4]:
print "loading data..."
x = cPickle.load(open("bigger_health-train-val-test2.pickle", "rb"))
revs, W, word_idx_map, vocab = x[0], x[1], x[2], x[3]
print "data loaded!"


datasets = make_idx_data(revs, word_idx_map, max_l=247, kernel_size=5)


loading data...
data loaded!


In [5]:
# Train data preparatio
N = datasets[0].shape[0]
conv_input_width = W.shape[1]
conv_input_height = int(datasets[0].shape[1]-1)

In [6]:
print "N: ", N
print "conv_input_height: ", conv_input_height
print "conv_input_width: ", conv_input_width

N:  27155
conv_input_height:  247
conv_input_width:  300


### Train Data Prepapration

In [7]:
# For each word write a word index (not vector) to X tensor
train_X = np.zeros((N, conv_input_height), dtype=np.int)
train_Y = np.zeros((N, 2), dtype=np.int)
for i in xrange(N):
    for j in xrange(conv_input_height):
        train_X[i, j] = datasets[0][i, j]
    train_Y[i, datasets[0][i, -1]] = 1
    
    
print 'train_X.shape = {}'.format(train_X.shape)
print 'train_Y.shape = {}'.format(train_Y.shape)

train_X.shape = (27155, 247)
train_Y.shape = (27155, 2)


In [8]:
print train_X[0] , type(train_X) , type(train_X[0])
print train_Y[0] , type(train_Y) , type(train_Y[0])

[30359 18897 32378  8607 30057 24613 35151 17688 13397   437 28698  7844
 23124  9241 32378 13814 28736 10380 30196     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [9]:
np.sum(train_Y) == train_Y.shape[0]

True

In [24]:
(np.sum(train_Y, axis=0)/float(np.sum(train_Y)))*100

array([ 31.1765789,  68.8234211])

### Prepare validation data

In [18]:
Nv = datasets[1].shape[0]

val_X = np.zeros((Nv, conv_input_height), dtype=np.int)
val_Y = np.zeros((Nv, 2), dtype=np.int)
for i in xrange(Nv):
    for j in xrange(conv_input_height):
        val_X[i, j] = datasets[1][i, j]
    val_Y[i, datasets[1][i, -1]] = 1
    
print 'val_X.shape = {}'.format(val_X.shape)
print 'val_Y.shape = {}'.format(val_Y.shape)

val_X.shape = (6845, 247)
val_Y.shape = (6845, 2)


In [19]:
print val_X[0] , type(val_X) , type(val_X[0])
print val_Y[0] , type(val_Y) , type(val_Y[0])

[13287   431 11571   112 16982  1204  1074 28736 15865 30613  6482 13814
 11571 24777 13194 20697 30352 31539 19006 31539 29072 13033 10002 23027
 31539 31983  2943 22730 16643 20903 16859 12153 27346 20722 31539 27346
 31983  2943  6892 28022 13683 22146 13287   431 17956 24441 23777  8607
 12326 28736 27346 18238 13814 19103 19016  9375 11034 18508 35234 13287
  9995 28374 20939  9375 30359 13479 17789 15865 27346 10898 19103  5248
 31540 12153 13814 11571  9713  8429 30359  1226 29976 27346 18897 11520
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [21]:
np.sum(val_Y) == val_Y.shape[0]

True

## Create a Keras model

In [25]:
# These will be used in the Embedding layers as paramteres
print "input_dim=", W.shape[0]
print "output_dim=", W.shape[1]
print "weights=", [W]
print "W_constraint=", unitnorm()

input_dim= 35972
output_dim= 300
weights= [array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  7.41578788e-02,  -1.86807379e-01,   1.21926129e-01, ...,
         -1.32260442e-01,   3.41992006e-02,   2.10501035e-04],
       [  1.43194227e-02,   6.02296554e-02,   2.19316810e-01, ...,
          8.92717242e-02,   4.24648672e-02,   2.71095056e-02],
       ..., 
       [ -2.34375000e-01,   2.14843750e-01,  -1.62109375e-01, ...,
          1.59179688e-01,   3.54003906e-02,  -1.13281250e-01],
       [ -1.21112600e-01,  -1.87677279e-01,  -2.32269242e-01, ...,
         -1.98359162e-01,  -1.15148671e-01,  -8.49112794e-02],
       [ -4.46777344e-02,   6.73828125e-02,  -1.41601562e-01, ...,
         -1.65039062e-01,   8.98437500e-02,  -5.51757812e-02]], dtype=float32)]
W_constraint= <keras.constraints.UnitNorm object at 0x5e3cd90>


In [48]:
model = Sequential()
model.add(Embedding(input_dim=W.shape[0], output_dim=W.shape[1], weights=[W], W_constraint=unitnorm()))

In [49]:
model.output_shape

(None, None, 300)

In [50]:
model.add(LSTM(250))
model.add(Activation('relu'))

In [51]:
model.output_shape

(None, 250)

In [52]:
model.add(Dense(2))
model.add(Activation('sigmoid'))

In [53]:
model.output_shape

(None, 2)

### Compile the model

In [54]:
#optimizer is adadelta and loss function is categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adadelta',
              metrics=['accuracy'])

### Fit the model

In [ ]:
model.fit(train_X, train_Y, batch_size=50, nb_epoch=10, validation_data=(val_X, val_Y))

Train on 27155 samples, validate on 6845 samples
Epoch 1/10
 2350/27155 [=>............................] - ETA: 640s - loss: 0.6337 - acc: 0.6774